In [7]:
import pandas as pd

# Corregir la ruta del archivo
df = pd.read_parquet("/home/donsson/proyectos/API/historico_ventas_1_año.parquet")
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet")


In [8]:
import re

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "CALI",
    "FMDE": "MEDELLIN",
    "FBOG": "BOGOTA",
    "FCTG": "CARTAGENA",
    "FBAQ": "BARRANQUILLA",
    "FVAL":"VALLADOLID"
}

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip().upper()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip().upper()
        else:
            # 3) Buscar prefijo de código (ej: FCAL, FMDE, etc.)
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # 🔹 Limpiar T1, T2, T3
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    return sucursal

# Aplicar
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

In [9]:
df = pd.merge(df,df_p,on='product_name',how="left")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773186 entries, 0 to 1773185
Data columns (total 10 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   price_subtotal           float64       
 1   id                       int64         
 2   quantity                 float64       
 3   product_id_num           int64         
 4   product_name             object        
 5   invoice_id_num           int64         
 6   invoice_name             object        
 7   date_invoice             datetime64[ns]
 8   Sucursal                 object        
 9   producto_costo_unitario  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 135.3+ MB


In [17]:
import pandas as pd
from datetime import date

# Semana y año actual
hoy = date.today()
semana_actual = 34  # lo defines manual si quieres
año_actual = hoy.year



# ================================
# 2) Agrupar ventas semanales
# ================================
df["Año"] = df["date_invoice"].dt.year
df["Semana"] = df["date_invoice"].dt.isocalendar().week

df_semana = (
    df.groupby(["Sucursal", "product_name", "Año", "Semana"])
      .agg({
          "quantity": "sum",
          "price_subtotal": "sum",
          "producto_costo_unitario": "mean"
      })
      .reset_index()
      .sort_values(["Sucursal", "product_name", "Año", "Semana"]) 
)

# ================================
# 3) Calcular ponderación personalizada
# ================================
# Asignar pesos: semanas 1-8 (80%), semanas 9-12 (20%)
df_semana = df_semana.sort_values(["Sucursal", "product_name", "Semana"])

def ema_custom(x):
    n = len(x)
    if n < 12:
        return None  # Si no hay 12 semanas completas
    ultimas = x.iloc[-4:]   # semanas 9-12
    anteriores = x.iloc[:-4]  # semanas 1-8
    return (ultimas.mean() * 0.2) + (anteriores.mean() * 0.8)

df_resultado = (
    df_semana.groupby(["Sucursal", "product_name"], as_index=False)
             .agg(EMA=("quantity", ema_custom),
                  Costo_unitario=("producto_costo_unitario", "mean"))
)


pd.set_option("display.max_colwidth", None) #muestra todo el contenido

# ================================
# 4) Agregar semana y año actuales
# ================================
df_resultado["Semana"] = semana_actual
df_resultado["Año"] = año_actual

# ================================
# 5) Calcular "Venta costo EMA"
# ================================
df_resultado["Venta_EMA"] = df_resultado["EMA"] * df_resultado["Costo_unitario"]

# ================================
# 6) Dejar columnas estilo Odoo
# ================================
df_resultado = df_resultado[[
    "Sucursal", "product_name", "Año", "Semana", "EMA",
    "Costo_unitario", "Venta_EMA"

]]


In [18]:
df_resultado = df_resultado.sort_values(by=("EMA"),ascending=False)

sede ="BARRANQUILLA"
df_sede = df_resultado[(df_resultado['Sucursal']==sede) ]
df_sede

,Sucursal,product_name,Año,Semana,EMA,Costo_unitario,Venta_EMA
733,BARRANQUILLA,"[DAB02570025] DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR. (025 DA2570)",2025,34,408.924000,13507.71,5.523627e+06
791,BARRANQUILLA,"[DAB04570025] DA4570 FILTRO AIRE 2. DONSSON - PERKINS, CATERPILLAR. (025 DA4570)",2025,34,350.884884,8840.52,3.102005e+06
752,BARRANQUILLA,"[DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITACHI (CARCASA GX2772A) (025 DA2772)",2025,34,301.894898,10680.97,3.224530e+06
562,BARRANQUILLA,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERPILLAR (125 B76)",2025,34,285.678000,32797.97,9.369658e+06
172,BARRANQUILLA,"[BCS00035125] GS035 FILTRO COMBUSTIBLE VOLVO,DEUTZ. (125 BF988)",2025,34,255.000000,20638.72,5.262874e+06
...,...,...,...,...,...,...,...
1503,BARRANQUILLA,[DLS00738189] GS738 FILTRO ACEITE - JAC (189 GS738),2025,34,NaN,9584.50,NaN
1504,BARRANQUILLA,[DLS00752189] GS752 FILTRO ACEITE - CUMMINS F2.5 (189 GS752),2025,34,NaN,20802.00,NaN
1505,BARRANQUILLA,"[DLS10286189] GS286A FILTRO ACEITE - CUMMINS ISM, ISX, X12, X15 (189 GS286A)",2025,34,NaN,26616.04,NaN
1506,BARRANQUILLA,[DRG0SOTR035] REFRIGERANTE ANTICONGELANTE SOT 50/50 HIDROCOOL x GALON (035 SOTR - GALON),2025,34,NaN,34634.94,NaN
